A EletroPlaza Store é um conglomerado de lojas presente em diversos países do mundo. Apesar da presença online da marca, as vendas são predominantemente físicas. A multinacional atua com eletrônicos, eletrodomésticos e acessórios em geral, oferecendo produtos acessíveis e de qualidade para uma vasta gama de clientes.

Após diversos investimentos realizados nos setores de vendas, marketing e desenvolvimento de produtos, a empresa teve um grande salto em seu faturamento no último ano. Porém, junto ao faturamento, vieram novos desafios em tentar entender precisamente como as vendas vão continuar se comportando nas próximas semanas.

O ano está chegando ao fim e a Black Friday e o Natal estão se aproximando. As vendas tendem a subir exponencialmente ainda mais nessa época e os dados de comparação do ano passado já não servem mais como termômetro para a empresa. Além disso, a equipe de negócio tem uma meta de faturamento e está preocupada se conseguirá alcançá-la. A urgência em entender os possíveis resultados é grande.

O desafio da sua equipe, formada por cientistas e analistas de dados, é construir uma solução que preveja as vendas semanais das lojas e de seus variados setores para as 5 semanas finais do ano. Deste modo, os gestores da EletroPlaza Store poderão definir planos futuros, organizar estoques, calcular receitas, decidir se farão novos investimentos ou não e onde devem alocar estes novos investimentos, através de campanhas mais eficientes de marketing e vendas.

Os dados disponíveis para a previsão são referentes às vendas realizadas do início do ano vigente até o momento. Cada linha representa um setor específico de uma loja específica naquela determinada semana. São fornecidas diversas informações para análise, inclusive dados externos como temperatura, preço de combustível e taxa de desemprego. Ainda, devido a urgência da solicitação, há diversos dados faltantes na base e a sua equipe não tem tempo hábil o suficiente para coletá-los, devendo buscar soluções alternativas para que mesmo assim consiga realizar previsões eficazes.

In [274]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import OneHotEncoder

from xgboost import XGBRegressor

## Load Data

In [275]:
path = '/home/edilson07/projects/hackday_7/data/'

train = pd.read_csv(path + 'train.csv')
train_features = pd.read_csv(path + 'train_features.csv')
stores = pd.read_csv(path + 'stores.csv')

test = pd.read_csv(path + 'test.csv')
test_features = pd.read_csv(path + 'test_features.csv')

df = pd.merge(train, train_features, on=['loja', 'data','feriado'], how='left').merge(stores, on='loja', how='left')
df_test = pd.merge(test, test_features, on=['loja', 'data','feriado'], how='left').merge(stores, on='loja', how='left')

In [276]:
df.head()

,id,loja,setor,data,vendas_semanais,feriado,temperatura,combustivel,desconto_1,desconto_2,desconto_3,desconto_4,desconto_5,desemprego,distancia_competidores,clientes,tipo,tamanho
0,140679,17,93,01-07,6283.00,nao,-14.316667,2.891,NaN,NaN,NaN,NaN,NaN,0.06866,NaN,541,eletronico,93188.0
1,140680,12,17,01-07,10006.77,nao,3.133333,3.287,NaN,NaN,NaN,NaN,NaN,0.14021,NaN,463,eletronico,112238.0
2,140681,3,25,01-07,2658.57,nao,11.861111,NaN,NaN,NaN,NaN,NaN,NaN,0.07551,NaN,901,eletronico,NaN
3,140682,42,1,01-07,8836.00,nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08744,NaN,176,outro,39690.0
4,140683,26,9,01-07,9526.27,nao,-6.038889,3.193,NaN,NaN,NaN,NaN,NaN,0.07907,NaN,124,eletrodomestico,152513.0


In [277]:
df['feriado'].unique()

array(['nao', 'sim'], dtype=object)

## Data Description

In [278]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135385 entries, 0 to 135384
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      135385 non-null  int64  
 1   loja                    135385 non-null  int64  
 2   setor                   135385 non-null  int64  
 3   data                    135385 non-null  object 
 4   vendas_semanais         122523 non-null  float64
 5   feriado                 135385 non-null  object 
 6   temperatura             87440 non-null   float64
 7   combustivel             82396 non-null   float64
 8   desconto_1              5926 non-null    float64
 9   desconto_2              5875 non-null    float64
 10  desconto_3              5639 non-null    float64
 11  desconto_4              5201 non-null    float64
 12  desconto_5              5926 non-null    float64
 13  desemprego              135385 non-null  float64
 14  distancia_competidor

## Check NaN

In [279]:
df.isna().sum()

id                             0
loja                           0
setor                          0
data                           0
vendas_semanais            12862
feriado                        0
temperatura                47945
combustivel                52989
desconto_1                129459
desconto_2                129510
desconto_3                129746
desconto_4                130184
desconto_5                129459
desemprego                     0
distancia_competidores    129459
clientes                       0
tipo                           0
tamanho                    33327
dtype: int64

In [280]:
def clean_nan(df):

    # descontos
    
    df['desconto_1'] = df['desconto_1'].fillna(0)
    df['desconto_2'] = df['desconto_2'].fillna(0)
    df['desconto_3'] = df['desconto_3'].fillna(0)
    df['desconto_4'] = df['desconto_4'].fillna(0)
    df['desconto_5'] = df['desconto_5'].fillna(0)
    
    # temperatura , combustivel ,tamanho - media
    df['temperatura'] = df['temperatura'].fillna(df['temperatura'].mean())
    df['combustivel'] = df['combustivel'].fillna(df['combustivel'].mean())
    df['tamanho'] = df['tamanho'].fillna(df['tamanho'].mean())
    
    # distancia_competidores, tamanho
    df['distancia_competidores'] = df['distancia_competidores'].fillna(df['distancia_competidores'].max())
    
    df = df.dropna()

    return df

df = clean_nan(df)
df_test = clean_nan(df_test)

## Change Types

In [281]:
def change_types(df):

    df['data'] = pd.to_datetime(df['data'], format='%m-%d')
    
    df['dia'] = df['data'].dt.day
    df['mes'] = df['data'].dt.month
    df['dia_semana'] = df['data'].dt.dayofweek
    df['trimestre'] = df['data'].dt.quarter
    df['semana_do_ano'] = df['data'].dt.isocalendar().week
    
    
    
    
    df = df.drop(['data'], axis=1)

    return df

df = change_types(df)
df_test = change_types(df_test)

# Pre Processing

## One Hot Encoding

In [282]:
feriados = {'nao' : 0,
            'sim': 1}

def make_feriados(feriados, df):

    df['feriado'] = df['feriado'].map(feriados)
    df = pd.get_dummies(df, prefix=['tipo'], columns=['tipo'])

    return df

df = make_feriados(feriados, df)
df_test = make_feriados(feriados, df_test)



# Machine Learning

In [254]:
X = df.drop(['vendas_semanais'], axis=1)
y = df['vendas_semanais']

In [285]:
xgb = XGBRegressor()

xgb.fit(X, y)

y_pred = xgb.predict(df_test)

# Submission

In [288]:
df_test['vendas_semanais'] = y_pred
df_submission = df_test[['id','vendas_semanais']]

In [290]:
df_submission.to_csv('primeira_submissao.csv', index=False)